In [6]:
import numpy as np
from PIL import Image
import os
import pandas as pd # Added pandas for robust CSV loading

# --- Configuration ---
# Input image path (ensure this file exists in your Kaggle input or working directory)
INPUT_IMAGE_PATH = '/kaggle/input/samplephoto/pexels-helenalopes-2253275.jpg'
# Output path for the hex representation of the image
OUTPUT_HEX_FILE_PATH = 'image_pixels_hex.hex' # Updated extension to .hex as per your log
# Path to the file containing integer weights (e.g., a 3x3 filter)
INTEGER_WEIGHTS_FILE_PATH = '/kaggle/input/intweights/VGG_Int8_Weights (4).csv'
# Output path for the convolved result (content will be text, not an actual image file)
CONVOLVED_OUTPUT_FILE_PATH = 'convolved_output_from_hex.txt' # Keeping as .txt as content is text

# Desired size for the image (resizing for consistent processing)
TARGET_IMAGE_SIZE = (50, 50) # Width, Height

# --- Helper Functions ---

def create_dummy_image(path, size=(50, 50)):
    """Creates a dummy grayscale image if one doesn't exist."""
    if not os.path.exists(path):
        print(f"Creating dummy image at {path}...")
        img = Image.new('L', size, color = 'grey') # 'L' for grayscale
        img.save(path)
        print("Dummy image created.")

def create_dummy_weights_file(path, weights_shape=(3, 3)):
    """Creates a dummy integer weights file (e.g., a 3x3 filter)."""
    if not os.path.exists(path):
        print(f"Creating dummy integer weights file at {path}...")
        # Example: a simple edge detection filter
        dummy_weights = np.array([
            [1, 0, -1],
            [2, 0, -2],
            [1, 0, -1]
        ], dtype=int)
        # Ensure the dummy weights match the desired shape
        if dummy_weights.shape != weights_shape:
            # If shape doesn't match, create random integers for the specified shape
            dummy_weights = np.random.randint(-5, 5, size=weights_shape)

        # Use np.savetxt for dummy file, assuming simple numerical format
        np.savetxt(path, dummy_weights, fmt='%d')
        print("Dummy weights file created.")

def image_to_hex_file(image_path, output_hex_path, target_size):
    """
    Converts an image to grayscale, resizes it, and saves its pixel values
    as a hex string in a file, formatted with line numbers.
    """
    print(f"\n--- Converting image '{image_path}' to hex format ---")
    try:
        # Open image and convert to grayscale ('L' mode)
        img = Image.open(image_path).convert('L')
        # Resize image to target size
        img = img.resize(target_size)
        # Get pixel data as a NumPy array
        pixel_array = np.array(img)

        with open(output_hex_path, 'w') as f:
            line_num = 1
            # Iterate through each pixel
            for pixel_value in pixel_array.flatten():
                # Convert pixel value (0-255) to 2-digit hexadecimal string
                hex_value = f"{pixel_value:02X}"
                # Write in the format: line_number hex_value
                f.write(f"{line_num}\t{hex_value}\n")
                line_num += 1
        print(f"Image successfully converted to hex and saved to '{output_hex_path}'.")
        return pixel_array # Return the pixel array for later use
    except FileNotFoundError:
        print(f"Error: Image file not found at '{image_path}'.")
        return None
    except Exception as e:
        print(f"An error occurred during image to hex conversion: {e}")
        return None

def load_integer_weights(weights_file_path):
    """
    Loads integer weights from a text file (CSV) into a NumPy array.
    This version specifically tries to extract a 3x3 kernel from the *start*
    of the flattened weights, assuming the CSV contains flattened weights.
    """
    print(f"\n--- Loading integer weights from '{weights_file_path}' ---")
    try:
        # Read the CSV, assuming it might have a header.
        # We'll read all columns as strings initially to avoid conversion errors on header.
        df = pd.read_csv(weights_file_path, header=None, dtype=str)

        # Find the first row that contains numerical data.
        # Iterate through rows and try to convert the first element to float.
        data_start_row = 0
        for i in range(len(df)):
            try:
                # Try converting the first element of the row to float.
                # This assumes the actual numerical data starts in the first column.
                _ = float(df.iloc[i, 0])
                data_start_row = i
                break
            except ValueError:
                continue
        
        if data_start_row > 0:
            print(f"Skipping {data_start_row} header row(s).")

        # Extract numerical data from the identified start row onwards, from the first column
        # Convert to a 1D numpy array of integers
        # Filter out any empty strings that might result from uneven rows or parsing
        all_weights_str = df.iloc[data_start_row:, 0].values.flatten()
        all_weights = [int(w) for w in all_weights_str if w.strip() != '']
        
        # VGG16 first conv layer kernel (3x3x3x64) has 1728 weights.
        # For manual convolution, we need a single 3x3 kernel.
        # We'll extract the first 9 weights and reshape them into a 3x3 kernel.
        # This assumes these correspond to a 3x3 filter for one input/output channel.
        if len(all_weights) < 9:
            raise ValueError(f"Not enough weights ({len(all_weights)}) found in the file to form a 3x3 kernel. Expected at least 9 numerical values.")

        # Extract the first 9 weights and reshape to 3x3
        kernel = np.array(all_weights[:9]).reshape((3, 3))
        
        print(f"Extracted 3x3 kernel from the beginning of the weights file.")
        print(f"Kernel shape: {kernel.shape}")
        return kernel
    except FileNotFoundError:
        print(f"Error: Weights file not found at '{weights_file_path}'.")
        return None
    except Exception as e:
        print(f"An error occurred during weights loading: {e}")
        # Provide more context if the error is a conversion issue
        if 'could not convert string' in str(e) and 'df' in locals():
            print(f"Problematic data might be around row {data_start_row}, column 0. First few values: {df.iloc[data_start_row:, 0].head().to_list()}")
        return None

def perform_manual_convolution(image_pixels, kernel, output_file_path):
    """
    Performs a manual 2D convolution operation (like a single CNN layer).
    Uses 'valid' padding (no padding, filter only operates where it fully fits).
    """
    print(f"\n--- Performing manual 2D convolution ---")
    if image_pixels is None or kernel is None:
        print("Cannot perform convolution: image pixels or kernel are missing.")
        return

    image_height, image_width = image_pixels.shape
    kernel_height, kernel_width = kernel.shape

    # Calculate output dimensions for 'valid' padding
    if image_height < kernel_height or image_width < kernel_width:
        print("Error: Image dimensions are smaller than kernel dimensions. Cannot convolve.")
        return

    output_height = image_height - kernel_height + 1
    output_width = image_width - kernel_width + 1

    # Initialize the output feature map
    output_feature_map = np.zeros((output_height, output_width), dtype=np.float32)

    print(f"Input Image Shape: {image_pixels.shape}")
    print(f"Kernel Shape: {kernel.shape}")
    print(f"Output Feature Map Shape: {output_feature_map.shape}")

    # Iterate over the output feature map dimensions
    for i in range(output_height):
        for j in range(output_width):
            # Extract the image patch
            image_patch = image_pixels[i : i + kernel_height,
                                       j : j + kernel_width]

            # Element-wise multiplication and summation
            convolved_value = np.sum(image_patch * kernel)
            output_feature_map[i, j] = convolved_value

    print("Manual convolution complete.")

    # Save the convolved output to a file
    try:
        with open(output_file_path, 'w') as f:
            f.write(f"Manual 2D Convolution Output (from hex-converted image)\n")
            f.write(f"Input Image Dimensions: {image_height}x{image_width}\n")
            f.write(f"Kernel Used:\n{kernel}\n")
            f.write(f"Output Feature Map Dimensions: {output_height}x{output_width}\n")
            f.write("-" * 50 + "\n")
            f.write(np.array_str(output_feature_map, precision=4, suppress_small=True))
            f.write("\n")
        print(f"Convolved output saved to '{output_file_path}'.")
    except Exception as e:
        print(f"Error saving convolved output to file: {e}")

# --- Main Execution ---
if __name__ == "__main__":
    # 1. Ensure dummy files exist for demonstration if not provided by user
    # These dummy files are created only if the specified paths don't exist.
    # In a Kaggle notebook, you'd typically use paths from /kaggle/input/
    create_dummy_image(INPUT_IMAGE_PATH, TARGET_IMAGE_SIZE)
    create_dummy_weights_file(INTEGER_WEIGHTS_FILE_PATH)

    # 2. Convert image to hex file format
    # This function also returns the pixel array for direct use in convolution
    image_pixels_array = image_to_hex_file(INPUT_IMAGE_PATH, OUTPUT_HEX_FILE_PATH, TARGET_IMAGE_SIZE)

    # 3. Load integer weights
    integer_weights_kernel = load_integer_weights(INTEGER_WEIGHTS_FILE_PATH)

    # 4. Perform manual convolution (simulating one layer)
    perform_manual_convolution(image_pixels_array, integer_weights_kernel, CONVOLVED_OUTPUT_FILE_PATH)

    print("\n--- Process Complete ---")
    print(f"Check '{OUTPUT_HEX_FILE_PATH}' for hex representation of the image.")
    print(f"Check '{CONVOLVED_OUTPUT_FILE_PATH}' for the result of the convolution (text content).")



--- Converting image '/kaggle/input/samplephoto/pexels-helenalopes-2253275.jpg' to hex format ---
Image successfully converted to hex and saved to 'image_pixels_hex.hex'.

--- Loading integer weights from '/kaggle/input/intweights/VGG_Int8_Weights (4).csv' ---
Skipping 1 header row(s).
Extracted 3x3 kernel from the beginning of the weights file.
Kernel shape: (3, 3)

--- Performing manual 2D convolution ---
Input Image Shape: (50, 50)
Kernel Shape: (3, 3)
Output Feature Map Shape: (48, 48)
Manual convolution complete.
Convolved output saved to 'convolved_output_from_hex.txt'.

--- Process Complete ---
Check 'image_pixels_hex.hex' for hex representation of the image.
Check 'convolved_output_from_hex.txt' for the result of the convolution (text content).
